# INSERT TITLE HERE

In the AV1 video coding format, a new entropy coder was added. An entropy
encoder is responsible for compressing the instructions that allow
reconstructing the video frame.

The advantage of the new entropy coder is that it uses a parallel method
capable of decoding more than two possibilities in one step. In comparison,
previous coders used a serial process that decoded in a binary manner. This was
a substantial bottleneck for hardware, which are able to utilize fine-grained
parallelization to a much greater extent than cpus or gpus. It's also possible
to write SIMD (TODO: link) for this process, so there can also be some benefits
for cpus.

To take advantage of this parallelism, data must be encoded from an 'alphabet'
of more than 2 'letters' or 'symbol'. The largest supported alphabet size is 16,
which allows processing 4-bit long letters. Each alphabet has an associated
distribution for each letter, $i$, stored and operated on in the form
$1 - CDF_i$ (in fixed-point arithmetic). This is an approximate of the frequency
that takes spatial locality into account instead of an exact model of frequency
like huffman coding. This structure for storing distributions will be referred
to as a CDF.

CDFs are generally updated whenever a symbol is encoded. Some CDFs are
hardcoded in the code, and there is also functionality to turn off updates
some period of time.

Frames can be split into 'tiles' with independent CDF 'contexts'. The tiles
in the next frame get their contexts from previous contexts or, alternatively,
resets to the initial cdfs.

In practice, AV1 doesn't take advantage of larger alphabets as much as one
would hope. In this tutorial, we will take a file containing a dump of a video's
symbol data and analyze the performance of the entropy encoder. We won't be
considering any video data that disable updates to CDFs or any of the hardcoded
CDFs.

TODO: Links https://jmvalin.ca/daala/revisiting/

## Getting the Data

libaom v3.1.0 was used to encode a clip using the following options
``aomenc -o <output>.ivf <input> --profile=0 --cpu-used=1 --threads=2 --tile-columns=2 --end-usage=q --cq-level=43``.
See ``aomenc --help`` for what these options do.

The clip used is ``Netflix_ToddlerFountain_4096x2160_60fps_10bit_420.y4m``
(can be found [here](https://media.xiph.org/video/derf/)) that has been
converted to 8 bits and downsampled to 1080p using ffmpeg.

To dump symbol data, modifications were made to the decoder in libaom. A branch
containing those changes can be found here (TODO: Link). Building the encoder
is broken on this branch, so it is necessary to run ``make aomdec`` instead of
just ``make`` to build successfully. The first 600 out of 1200 frames were
decoded using ``../aomdec <input>.ivf -o /dev/null --progress -t 1 --limit=600``.

The resulting output can be found here (TODO: Link)

### Format
The file contains a list of events and data for each event.

Events:
- The initial cdf and some descriptors for the event. Descriptors include:
    - An id to uniquely identify the individual cdf.
    - The function name of the CDF, or a custom name already provided by libaom.$^*$
    - src file and line number that the symbol first encoded on.$^*$
- Encode a symbol with update.
- Encode a symbol without update (not supported).
- Move CDFs from one context to another.
-

The

$^*$ Unfortunately, it might be possible for these to be different for one id
with two different files.

In [ ]:
import pandas as pd
import numpy as np
%load_ext Cython
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%%cython

# Tree that demonstrates the movement of contexts.
cdef class ContextTree:
    # Starting context or the
    cdef public int root_context
    # dict mapping to children that copy from a given context
    # list of  = dict[parent]
    cdef public dict children
    cdef public dict parents

    def __init__(self):
        self.root_context = 0
        self.children = {self.root_context:[]}
        self.parents = {}

    def chain_contexts(self, parent, child):
        self.parents[child] = parent
        self.children[parent].append(child)
        self.children.setdefault(child, [])

import numpy as np
cimport numpy as np
from libc.stdio cimport FILE, fopen, fclose, fgetc, fread, feof

# Read little endian 16-bit unsigned integer
cdef inline int read_u16_le(unsigned char* buf):
    return buf[0] | (buf[1] << 8)

cdef inline str fread_str(FILE* file, unsigned char* buf):
    fread(buf, 1, 1, file)
    strlen = buf[0]
    fread(buf, 1, strlen, file)
    return buf[:strlen].decode()

def cython_load_symbol_file(char* filename):
    cdef int event_type
    cdef char symbol
    cdef unsigned char buf[255]
    cdef FILE* file = fopen(filename, 'rb')

    cdef list df_name_id = []
    cdef dict symbol_data = {}
    cdef dict initial_cdfs = {}
    cdef context_tree = ContextTree()

    cdef dict current_context = None

    # Cache the current cdf for extra performance
    # TODO: change name
    cdef int current_cdf_id = -1
    cdef bytearray current_cdf_symbols = None

    # Loop through each event,
    while True:
        event_type = fgetc(file)
        if event_type == -1: # EOF
            break
        elif event_type == 0: # initialize defaults
            fread(buf, 1, 3, file)
            (cdf_id, num_symbols) = read_u16_le(buf), buf[2]
            name = fread_str(file, buf)
            #fread(buf, 1, strlen, file)
            #name = buf[:strlen].decode()
            src_file = fread_str(file, buf)
            fread(buf, 1, 2, file)
            src_line = read_u16_le(buf)
            # Remove full path
            src_file = src_file[src_file.rfind('/')+1:]
            df_name_id.append([cdf_id, name, src_file, src_line])

            initial_cdf = np.zeros(num_symbols, np.uint16)
            for i in range(num_symbols):
                fread(buf, 1, 2, file)
                initial_cdf[i] = read_u16_le(buf)
            initial_cdfs[cdf_id] = initial_cdf
        elif event_type == 1:
            # read symbol with a context update
            fread(buf, 1, 3, file)
            symbol, cdf_id = buf[0], read_u16_le(buf + 1)
            if cdf_id != current_cdf_id:
                current_cdf_id = cdf_id
                current_cdf_symbols = current_context.setdefault(cdf_id, bytearray())
            current_cdf_symbols.append(symbol)
        elif event_type == 2:
            # read symbol without updating context
            print('Symbols encoded without cdf update not supported')
        elif event_type == 3:
            # copy a previous context into this one
            fread(buf, 1, 4, file)
            parent_context_timestamp = read_u16_le(buf)
            child_context_timestamp = read_u16_le(buf + 2)
            context_tree.chain_contexts(
                parent_context_timestamp, child_context_timestamp
            )
        elif event_type == 4:
            # change the current context

            # read what
            fread(buf, 1, 2, file)
            context_timestamp = read_u16_le(buf)

            # This will probably never reuse a context
            current_context = symbol_data.setdefault(context_timestamp, {})

            # Invalidate the cdf cache
            current_cdf_id = -1
            current_cdf_symbols = None
        else:
            print('Unrecognized event read: %d'% event_type)
            break
    fclose(file)

    return symbol_data, context_tree, df_name_id, initial_cdfs

In [ ]:
def load_symbol_file(filename):
    symbol_data, context_tree, df_name_id, initial_cdfs = cython_load_symbol_file(filename)
    #size = 0
    for x in symbol_data.values():
        for (a, b) in x.items():
            x[a] = np.array(b, dtype=np.uint8)
            # size+=len(x[a])
    #np.savez('tmp.npz', symbol_data)
    #symbol_data = np.load('tmp.npz', allow_pickle=True)
    # tmp = open("tmp.pickle", "wb") # TODO: stop writing to ssd
    # pickle.dump(symbol_data, tmp)
    # tmp.close()
    #
    # # Defragment memory???
    # tmp = open("tmp.pickle", "rb")
    # symbol_data = pickle.load(tmp)
    # tmp.close()

    df_cdfs = pd.DataFrame(
        df_name_id,
        columns=['id', 'descriptor', 'src_file', 'line']
    )
    # TODO: Join for each sample.

    return symbol_data, context_tree, df_cdfs, initial_cdfs

start_time = datetime.datetime.now()

# TODO: rename df_name_id
symbol_data, context_tree, df_name_id, initial_cdfs = load_symbol_file(b"/home/kyle/Programming/VideoEncode/libaom/events.bin")

end_time = datetime.datetime.now()
print('duration', end_time - start_time)

In [ ]:
%%cython

import numpy as np
cimport numpy as np

#def symbol_bits(np.ndarray[np.uint16_t, ndim=1] cdf, int symbol):
#    cdf[symbol]
#    pass

# Holds an initial count variable
cdef class CDFState:
    cdef public np.ndarray cdf
    # Number of symbols encoded so far
    cdef public int count

    cdef int min_prob
    cdef float base_prob

    def __init__(self, cdf, count = 0):
        self.cdf = cdf.copy()
        self.count = count

        self.min_prob = 4
        # TODO: Should be close to right, need to verify
        self.base_prob = np.log2((1<<15) + self.min_prob*len(cdf))

    # def transfer(self):
    #     # In av1, using a previous context always resets the count
    #     return CDFState(self.cdf, count=0)

    # See https://github.com/xiph/rav1e/blob/c5bc3402475bf6b61907cfc0d73574caaaee3be9/src/ec.rs#L894
    def update(self, int symbol):
        cdef np.ndarray[np.uint16_t, ndim=1] cdf = self.cdf
        cdef int nsymbols = len(cdf)
        cdef int rate = 3 + min((nsymbols + 1) >> 1, 2)

        rate += (self.count >> 4)
        self.count += 1 - (self.count >> 5)

        for i in range(nsymbols):
            if i >= symbol:
                cdf[i] -= cdf[i] >> rate
            else:
                cdf[i] += ((1 << 15) - cdf[i]) >> rate

    def calc_symbol_cost(self, int symbol):
        cdef np.ndarray[np.uint16_t, ndim=1] cdf = self.cdf
        a = 1<<15
        if symbol != 0:
            a = int(cdf[symbol - 1])
        diff = a - cdf[symbol]
        return self.base_prob - np.log2(float(diff + self.min_prob))

In [ ]:
import scipy.stats
import scipy.signal
import scipy.ndimage
from statsmodels.tsa.stattools import adfuller, kpss

def calc_symbol_cost(cdf_state, symbol):
    nsymbols = len(cdf_state.cdf)
    a = 1<<15
    if symbol != 0:
        a = int(cdf_state.cdf[symbol - 1])
    diff = a - cdf_state.cdf[symbol]
    min_prob = 4
    # TODO: Should be close to right, need to verify
    return np.log2(
        ((1<<15) + min_prob*nsymbols)/(diff + min_prob)
    )

def data_entropy(data):
    return scipy.stats.entropy(
        np.unique(data,return_counts=True,axis=0)[1], axis=0, base=2
    )

class SymbolCounters:
    def __init__(self, size):
        self.counts = np.zeros(size)

    def count_symbols(self, symbols):
        #indices, counts_scatter = np.unique(symbols, return_counts=True)
        #self.counts[indices] += counts_scatter
        self.counts += np.bincount(symbols, minlength=len(self.counts))

    def entropy(self):
        return scipy.stats.entropy(self.counts, base=2)

    def to_ec_cdf(self):
        cdf = 1 - (self.counts / self.counts.sum()).cumsum()
        return (cdf * (1<<15)).astype(np.uint16)

def iter_contexts(tree):
    path = [tree.root_context]
    while len(path) != 0:
        parent = path.pop()
        for child in tree.children[parent]:
            #symbols = symbol_data.get(child) # TODO: purge empty???
            #if symbols is not None:
            yield parent, child
            path.append(child)

#for p, c in iter_contexts(context_tree):
#    print(p, c)

In [ ]:
start_time = datetime.datetime.now()

df_stats = pd.DataFrame(
    columns=[
        'id',
        'descriptor',
        'src_file',
        'line',
        'alphabet_size',
        'bits',
        'num_symbols',
        'bits_per_symbol',
        'bits_over_entropy',
        #'bits_saved_vs_entropy'
    ]
)

total_cost = 0
total_entropy = 0
context_end_cdfs = {context_tree.root_context: initial_cdfs}
for cdf_id, initial_cdf in initial_cdfs.items():
    new_row = dict()
    # TODO: JUST COPY FROM TABLE
    new_row['id'] = cdf_id
    new_row['descriptor'] = df_name_id[df_name_id.id == cdf_id].descriptor.tolist()[0]
    new_row['src_file'] = df_name_id[df_name_id.id == cdf_id].src_file.tolist()[0]
    new_row['line'] = df_name_id[df_name_id.id == cdf_id].line.tolist()[0]
    new_row['alphabet_size'] = len(initial_cdf)
    num_symbols = 0
    cost = 0
    count = SymbolCounters(len(initial_cdf))

    for parent_context, child_context in iter_contexts(context_tree):
        cdf = context_end_cdfs[parent_context][cdf_id]
        context_end_cdfs.setdefault(child_context, {})[cdf_id] = cdf
        context = symbol_data[child_context]
        if context is None:
            continue
        #if child_context != 1:
        #    continue

        arr = context.get(cdf_id)
        if arr is not None:# and len(arr) > 100:# and cdf_id == 2520:
            num_symbols += len(arr)

            cdf_state = CDFState(cdf)

            count.count_symbols(arr)
            # static_matrix = np.repeat(count.counts[:,None], len(cdf), axis=1)
            # static_matrix = static_matrix @ static_matrix.T
            # static_matrix /= static_matrix.sum()


            #test_cdf = count.to_ec_cdf()
            #cdf_state = CDFState(test_cdf, count=32)


            # Quasi-Markov Chain
            # tmp = arr.view()[0:len(arr) >> 1 << 1]
            # tmp = tmp.reshape((len(tmp) >> 1, 2))
            # indices, counts_scatter = np.unique(tmp, return_counts=True, axis=0)
            #print(len(initial_cdf))
            #print(scipy.stats.entropy(counts_scatter, base=2)/2)

            # matrix = np.zeros((len(cdf), len(cdf)))
            # matrix[indices[:,0], indices[:,1]] += counts_scatter
            # matrix /= matrix.sum() / (1 << 15)


            cdf_time_series = []
            for symbol in arr:
                #cost += calc_symbol_cost(cdf_state, symbol)
                cost += cdf_state.calc_symbol_cost(symbol)
                cdf_state.update(symbol)
                #cdf_time_series.append(cdf_state.cdf.copy())
            context_end_cdfs[child_context][cdf_id] = cdf_state.cdf
            #print(cdf_state.cdf)

            # if len(cdf) <= 4:
            #     cdf = -np.diff(cdf, prepend=(1<<15)).astype(float)
            #     static_matrix = np.repeat(cdf[:,None], len(cdf), axis=1)
            #     static_matrix = (static_matrix @ static_matrix.T).flatten()
            #     static_matrix /= static_matrix.sum()
            #     cdf = static_matrix
            #     cdf = ((1 - cdf.cumsum()) * (1<<15)).astype(np.uint16)
            #
            #     tmp = tmp @ np.array([1,len(cdf)])
            #
            #     cdf_state = CDFState(cdf)
            #     costW = 0
            #     for symbol in tmp:
            #         costW += calc_symbol_cost(cdf_state, symbol)
            #         cdf_state.update(symbol)
            #     print(costW/len(arr))
                #print(cdf)


            # cdf_time_series = np.array(cdf_time_series, dtype=float)
            # #cdf_time_series -= cdf_time_series.mean(axis=0)
            # cdf_time_series = -np.diff(cdf_time_series, axis=1, prepend=(1<<15))
            # cdf_time_series = cdf_time_series.T


            # cdf_time_series = scipy.signal.convolve2d(
            #     cdf_time_series,
            #     np.ones((1,64))/64,
            #     mode='valid'
            # )
            # cdf_time_series = scipy.ndimage.gaussian_filter1d(
            #     cdf_time_series,
            #     sigma = 15,
            #     axis=1
            # )


            # adfs = []
            # adf_pvals = []
            # for series in cdf_time_series:
            #     adf, pval = adfuller(series)[:2]
            #     adfs.append(adf)
            #     adf_pvals.append(pval)
                #print(adf, pval)
                #print(adfuller(series, regresults=True)[-1].resols.summary())
                # if not np.isnan(adf):
                #     kpss_stat, pvalue = kpss(series, nlags='auto')[:2]
                #     print(kpss_stat, pvalue)

            #print(df_name_id[df_name_id.id == cdf_id].descriptor.tolist())
            # print('ID: %5d'%cdf_id+'\tLevels: %d'%len(cdf_state.cdf)+
            #       '\tbits: %.2f'% cost+'\tsymbols: %d'% len(arr),
            #       '\tbits/symbol: %.2f'%avg_cost+
            #       ' entropy: %.2f'%entropy+
            #       ' diff: %.2f'%entropy_diff)
            #print()
    if num_symbols == 0:
        continue
    new_row['num_symbols'] = num_symbols

    total_cost += cost
    new_row['bits'] = cost # bit cost???
    avg_cost = cost/num_symbols
    new_row['bits_per_symbol'] = avg_cost

    entropy = count.entropy()
    new_row['entropy'] = entropy

    #entropy_diff = entropy - avg_cost
    #bits_saved = entropy_diff * num_symbols
    #new_row['bits_saved_vs_entropy'] = bits_saved

    entropy_ratio = np.nan
    if entropy != 0:
        entropy_ratio = avg_cost / entropy
    new_row['bits_over_entropy'] = entropy_ratio

    total_entropy += entropy*num_symbols

    df_stats = df_stats.append(new_row, ignore_index=True)

# TODO: Might not want to display bits_per_symbol. Nah. Keep it.
# TODO: Rename this monstrosity avg_cost/bits_over_alphabet_size
df_stats['bits_per_symbol_over_alphabet_size'] = df_stats.bits_per_symbol / df_stats.alphabet_size

print(total_cost)
print(total_entropy)
end_time = datetime.datetime.now()
print('duration', end_time - start_time)

# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[0])
# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[1])
# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[2])
# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[3])
#plt.show()

#df_stats.sort_values('bits_saved_vs_entropy')[:50]
#df_stats[[x[0][0] == 'read_mv_component' for x in df_stats.descriptors]].sort_values('bits', ascending=False)[:35]
#df_stats.sort_values('bits_per_choice', ascending=False)[0:25]
pd.options.display.float_format = '{:,.3f}'.format
df_stats.sort_values('bits', ascending=False)

Lots of duplicate of src + line. Want some concept of what is happening with
them combined. DropNA???

In [ ]:
# TODO: Use as a check

# TODO: fix me earlier
#df_stats.bits_per_symbol_over_alphabet_size = df_stats.bits_per_symbol_over_alphabet_size.apply(pd.to_numeric)
#df_stats.alphabet_size = df_stats.alphabet_size.apply(pd.to_numeric)

#group_stats = df_stats.dropna().groupby(['descriptor', 'src_file', 'line'])
group_stats = df_stats.groupby(['descriptor', 'src_file', 'line'])
combined = group_stats.agg(
    count = ('id', 'count'),
    bits = ('bits', 'sum'),
    num_symbols = ('num_symbols', 'sum'),
    # Weighted average
    bits_per_symbol_over_alphabet_size = (
            'bits_per_symbol_over_alphabet_size',
            lambda x: np.average(x, weights=df_stats.loc[x.index, "num_symbols"])
        )
).reset_index()
combined.sort_values(by='bits', ascending=False)

In [ ]:
combined.sort_values(by='bits', ascending=False)[0:25]

In [ ]:
combined.sort_values(by='bits_per_symbol_over_alphabet_size', ascending=False)[0:25]

In [ ]:
combined.filter(regex='read_mv', axis = 0)
combined[combined.descriptor.str.match('read_mv')]

In [ ]:
mv_bits_stats = df_stats[df_stats.descriptor.str.match('read_mv') & (df_stats.line == 858)][
    ["id", "bits", "num_symbols", "bits_per_symbol_over_alphabet_size"]
]
mv_bits_stats

There should be enough symbols to form a reasonable confidence level for all of these.

In [ ]:
sns.swarmplot(x=mv_bits_stats.bits_per_symbol_over_alphabet_size)
plt.show()

In [ ]:
# TODO: Move up earlier
#sns.swarmplot(
sns.violinplot(
    x=df_stats[df_stats.num_symbols > 30].bits_per_symbol_over_alphabet_size,
#    x=df_stats[df_stats.num_symbols > 50].bits_per_symbol_over_alphabet_size,
#    size=2
    cut=0
)
plt.show()

#df_stats[df_stats.num_symbols > 30].bits.to_numpy()

#df_stats[df_stats.num_symbols > 30].alphabet_size.to_numpy()

In [ ]:
#subset = df_stats.filter()

# Too many items
df_stats.sort_values('bits_per_symbol_over_alphabet_size', ascending=False)

In [ ]:
# group_stats.filter(lambda x: len(x) > 15).boxplot(
#     by=['descriptor', 'src_file', 'line'],
#     column='bits_per_symbol_over_alphabet_size',
#     #rot,
#     vert = False
# )
# plt.show()

#group_stats.filter(lambda x: len(x) > 100)

#combined.sort_values('count', ascending=False)

Filter out entries with small cost or few symbols.

Maybe do work on

TODO: expand

In [ ]:
# df_stats[df_stats.num_symbols > 200].sort_values(
#     'bits_per_symbol_over_alphabet_size',
#     ascending=False
# )

In [ ]:
# df_stats[
#     df_stats.num_symbols > 100
# ].sort_values(
#     'bits_per_symbol_over_alphabet_size',
#     #'bits_over_entropy',
#     ascending=False
# ).to_csv("stats.csv")

In [ ]:
# test = df_stats[['src_file', 'line']]
# test = test.value_counts()
# test

In [ ]:
heights = df_stats[df_stats.bits > 1000].sort_values(
    'bits_over_entropy', ascending=False
).bits_over_entropy.to_numpy()
print(len(heights))

plt.bar(
    np.arange(len(heights)),
    heights-1,
    bottom=1
)
plt.show()

# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[0])
# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[1])
# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[2])
# plt.plot(np.arange(cdf_time_series.shape[1]), cdf_time_series[3])
#plt.show()

In [ ]:
# TODO: 100% should use seperate columns for line/src
# TODO: ditch
id_joint = 4055
id_mvsign0 = 4060
id_mvsign1 = 4078
id_mvclass0 = 4056
id_mvclass1 = 4074
joint_symbols = context[id_joint]
mvsign0 = context[id_mvsign0]
mvsign1 = context[id_mvsign1]
mvclass0 = context[id_mvclass0]
mvclass1 = context[id_mvclass1]
print(np.bincount(joint_symbols))
print(len(mvsign0))
print(len(mvsign1))
mv0_index = 0
mv1_index = 0
matrix = []
for x in joint_symbols:
    if x == 0:
        matrix.append([0, 0])
    elif x == 1:
        matrix.append([0, mvclass1[mv1_index] + 1])
        mv1_index += 1
    elif x == 2:
        #matrix.append([mvclass0[mv0_index] > 1, 0])
        matrix.append([mvclass1[mv0_index] + 1, 0])
        mv0_index += 1
    elif x == 3:
        #matrix.append([mvclass0[mv0_index] > 1, 1])
        matrix.append([mvclass0[mv0_index] + 1, mvclass1[mv1_index] + 1])
        mv0_index += 1
        mv1_index += 1
#print(mvclass0)
print(
    len(joint_symbols) * data_entropy(joint_symbols) +
    len(mvclass0) * data_entropy(np.minimum(mvclass0,2)) +
    len(mvclass1) * data_entropy(np.minimum(mvclass1,2)))
print(len(matrix) * data_entropy(np.minimum(np.array(matrix), 3)))

#print(df_name_id.id.unique().max())

In [ ]:
#adf, pvalue = adfuller((np.random.random(1000)-.5).cumsum())[:2]
#print(adf, pvalue)

# gen = (np.random.random(10000) - 0.5).cumsum()
# adf, pvalue = adfuller((np.random.random(1000)-.5).cumsum())[:2]
# print(adf, pvalue)
# kpss_stat, pvalue = kpss(gen, nlags='auto')[:2]
# print(kpss_stat, pvalue)
# plt.plot(np.arange(len(gen)), gen)
# plt.show()

#resstore = adfuller(series, store=True)[-1]
#resstore.H0

# for k,v in initial_cdfs.items():
#     print("%s: %s"%(df_name_id[df_name_id.id == k].descriptor.tolist(), v))

In [ ]:
#cdf_id = df_name_id.id.unique()[8]
#total_counts = np.zeros(16)
# for x in symbol_data.values():
#     arr = x.get(cdf_id)
#     if arr is not None:
#         (indices, counts) = np.unique(arr, return_counts=True)
#         total_counts[indices] += counts

# (indices, counts) = np.unique(x, return_counts=True)
# total_counts[indices] += counts
# print(total_counts)
#
# import scipy.stats
# scipy.stats.entropy(total_counts, base=2)

```c
842|static int read_mv_component(aom_reader *r, nmv_component *mvcomp,
843|                             int use_subpel, int usehp) {
844|  int mag, d, fr, hp;
845|  const int sign = aom_read_symbol(r, mvcomp->sign_cdf, 2, ACCT_STR);
846|  const int mv_class =
847|      aom_read_symbol(r, mvcomp->classes_cdf, MV_CLASSES, ACCT_STR);
848|  const int class0 = mv_class == MV_CLASS_0;
849|
850|  // Integer part
851|  if (class0) {
852|    d = aom_read_symbol(r, mvcomp->class0_cdf, CLASS0_SIZE, ACCT_STR);
853|    mag = 0;
854|  } else {
855|    const int n = mv_class + CLASS0_BITS - 1;  // number of bits
856|    d = 0;
857|    for (int i = 0; i < n; ++i)
858|      d |= aom_read_symbol(r, mvcomp->bits_cdf[i], 2, ACCT_STR) << i;
859|    mag = CLASS0_SIZE << (mv_class + 2);
860|  }
861|
862|  if (use_subpel) {
863|    // Fractional part
864|    fr = aom_read_symbol(r, class0 ? mvcomp->class0_fp_cdf[d] : mvcomp->fp_cdf,
865|                         MV_FP_SIZE, ACCT_STR);
866|
867|    // High precision part (if hp is not used, the default value of the hp is 1)
868|    hp = usehp ? aom_read_symbol(
869|                     r, class0 ? mvcomp->class0_hp_cdf : mvcomp->hp_cdf, 2,
870|                     ACCT_STR)
871|               : 1;
872|  } else {
873|    fr = 3;
874|    hp = 1;
875|  }
876|
877|  // Result
878|  mag += ((d << 3) | (fr << 1) | hp) + 1;
879|  return sign ? -mag : mag;
880|}
881|
882|static INLINE void read_mv(aom_reader *r, MV *mv, const MV *ref,
883|                           nmv_context *ctx, MvSubpelPrecision precision) {
884|  MV diff = kZeroMv;
885|  const MV_JOINT_TYPE joint_type =
886|      (MV_JOINT_TYPE)aom_read_symbol(r, ctx->joints_cdf, MV_JOINTS, ACCT_STR);
887|
888|  if (mv_joint_vertical(joint_type))
889|    diff.row = read_mv_component(r, &ctx->comps[0], precision > MV_SUBPEL_NONE,
890|                                 precision > MV_SUBPEL_LOW_PRECISION);
891|
892|  if (mv_joint_horizontal(joint_type))
893|    diff.col = read_mv_component(r, &ctx->comps[1], precision > MV_SUBPEL_NONE,
894|                                 precision > MV_SUBPEL_LOW_PRECISION);
895|
896|  mv->row = ref->row + diff.row;
897|  mv->col = ref->col + diff.col;
898|}
```